In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [8]:
df = pd.read_csv("../TheOtherOption/data/all_products_ah_nutr.csv", index_col=0)

In [9]:
df.head()

,allergy,id,brand,category,description,features,image_url,ingredients,nutrition,summary,...,Vitamine B12 / Cyano-cobalamine_val,Vitamine B12 / Cyano-cobalamine_unit,Vitamine D_val,Vitamine D_unit,Vitamine H / Biotine_val,Vitamine H / Biotine_unit,Waarvan zetmeel_val,Waarvan zetmeel_unit,Oplosbare voedingsvezel_val,Oplosbare voedingsvezel_unit
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,no_info,wi54074,AH,"Aardappel, groente, fruit/Komkommer (vers)",AH Komkommer,"['vegetarian', 'free_from_gluten', 'free_from_...",https://static.ah.nl/static/product/AHI_434d50...,no info,no info,"Komkommer heeft een frisse, knapperige, neutra...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,no_info,wi197393,AH,"Aardappel, groente, fruit/Bananen",AH Bananen,"['ah_vegan', 'free_from_milk', 'free_from_glut...",https://static.ah.nl/static/product/AHI_435452...,no info,"{'Eiwitten': '1 g', 'Energie': '385 kJ (91 kca...",Bananen met een zachte en zoete smaak. Lekker ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,no_info,wi4164,AH,"Aardappel, groente, fruit/Courgette (vers)",AH Courgette,"['vegetarian', 'free_from_milk', 'free_from_gl...",https://static.ah.nl/static/product/AHI_434d50...,no info,no info,"De courgette heeft een frisse, zachte, redelij...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,no_info,wi41194,AH,"Aardappel, groente, fruit/Paprika mix (vers)",AH Paprika mix,"['ah_vegan', 'free_from_gluten', 'free_from_mi...",https://static.ah.nl/static/product/AHI_435452...,no info,"{'Eiwitten': '0.8 g', 'Energie': '100 kJ (24 k...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,no_info,wi4177,AH,"Aardappel, groente, fruit/Broccoli (vers)",AH Broccoli,NaN,https://static.ah.nl/static/product/AHI_435452...,no info,no info,"Broccoli heeft een knapperige, frisse iets pit...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
allergies = ['selderij', 'lactose', 'melk. Kan bevatten: eieren', 'vis', 'soja']
allergy_list = []
kan_bevatten_list = []
tag = ""
for a in allergies:
    if tag != "kb":
        if "Kan bevatten" in a:
            tag = "kb"
            new_a = a.split(".")
            for b in new_a:
                if "Kan bevatten" in b:
                    new_b = b.split(":")
                    kan_bevatten_list.append(new_b[1].strip())
                else:
                    allergy_list.append(b.strip())
        else:
            allergy_list.append(a)
    else:
        kan_bevatten_list.append(a)
print(allergy_list)
print(kan_bevatten_list)

['selderij', 'lactose', 'melk']
['eieren', 'vis', 'soja']


In [92]:
df_allergies = df.copy()
all_allergies = []
for ind, allergies in df["allergy"].items():
    allergy_list = []
    kan_bevatten_list = []
    tag = ""
    if allergies == "no_info":
        df_allergies.loc[ind, "allergies"] = "no_info"
        df_allergies.loc[ind, "allergies_kan_bevatten"] = "no_info"
    else:
        for a in eval(allergies):
            if tag != "kb":
                if "Kan bevatten" in a:
                    tag = "kb"
                    new_a = a.split(".")
                    for b in new_a:
                        if "Kan bevatten" in b:
                            new_b = b.split(":")
                            kan_bevatten_list.append(new_b[1].strip())
                            all_allergies.append(new_b[1].strip())
                        else:
                            allergy_list.append(b.strip())
                            all_allergies.append(b.strip())
                else:
                    allergy_list.append(a)
                    all_allergies.append(a)
            else:
                kan_bevatten_list.append(a)
                all_allergies.append(a)
        if kan_bevatten_list == []:
            kan_bevatten_list = "no_info"
        df_allergies.at[ind, "allergies_kan_bevatten"] = kan_bevatten_list
        df_allergies.at[ind, "allergies"] = allergy_list

{'amandel',
 'cashewnoot',
 'eieren',
 'gerst',
 'glutenbevattende granen',
 'haver',
 'hazelnoot',
 'lactose',
 'macadamianoot',
 'melk',
 'mosterd',
 'noten',
 'paranoot',
 'pecannoot',
 "pinda's (aardnoten)",
 'pistache-noot',
 'selderij',
 'sesamzaad',
 'soja',
 'spelt',
 'sulfieten',
 'tarwe',
 'vis',
 'walnoot'}

In [105]:
for allergy in all_allergies:
    df_allergies[allergy] = ""

In [106]:
for ind, allergy in df_allergies["allergies"].items():
    if allergy != "no_info":
        for a in allergy:
            df_allergies.loc[ind, a] = "allergy"

In [113]:
for ind, allergy in df_allergies["allergies_kan_bevatten"].items():
    if allergy != "no_info":
        for a in allergy:
            df_allergies.loc[ind, a] = "kan bevatten"

In [119]:
df_allergies.to_csv("../TheOtherOption/data/all_products_ah_nutr_allergies.csv", index=False)